# 09. 폼과 로그인 뚫기

웹 스크레이핑의 기본을 이해한 다음 첫 질문은 로그인해야 얻을 수 있는 정보에 어떻게 접근하냐-!

앞의 장에서는 대부분 HTTP GET 을 사용했는데 이번 장에서는 웹 서버에서 저장하고 분석할 정보를 보내는 POST 메서드에 집중!

폼은 기본적으로 웹 서버가 이해하고 사용할 수 있는 POST 요청을 사용자가 보낼 수 있게 하는 수단-!

## requests 라이브러리
- 복잡한 HTTP 요청과 쿠키, 헤더를 아주 잘 처리하며 기타 기능이 있음

## 기본적인 폼 전송
- 대부분의 웹 폼은 HTML 필드와 전송 버튼, 폼을 실제로 처리하는 '액션'페이지로 구성됩니다.
- 보통 HTML 필드는 텍스트 필드이지만, 파일 업로드라든가 그 외 텍스트가 아닌 필드도 존재

http://pythonscraping.com/pages/files/form.html


- 폼의 동작을 흉내내려 한다면 변수 이름을 정확히 맞춰야 한다-!
- 개발자 도구를 활용해 코드를 분석해본 후, 실제 동작하는 곳을 찾아보기 ( header의 From Data 부분을 보자 )

In [3]:
import requests
params = {"firstname":"Ryan","lastname":"Matchell"}
r = requests.post("http://pythonscraping.com/pages/files/processing.php", data=params)
print(r.text)

Hello there, Ryan Matchell!


## 라디오 버튼, 체크박스, 기타 필드

- 라디오 버튼, 체크박스, 셀렉트 박스 등 다양한 입력 필드가 존재함. 
- 사실 신경 쓸 것은 필드 이름과 값뿐!

## 파일과 이미지 전송
- 결국 requests를 사용함!

In [4]:
files = {'uploadFile': open('logo.png', 'rb')}
r = requests.post("http://pythonscraping.com/pages/processing2.php", files=files)
print(r.text)

The file logo.png has been uploaded. <a href="/pages/uploads/logo.png">Link</a>


## 로그인과 쿠키 처리
- 최신 웹사이트는 대부분 쿠키를 사용해 누가 로그인했고 누가 안 했는지 추적함.
- 로그인 요청을 인증하면, 사이트는 브라우저에 쿠키를 저장함 ( 생성한 토큰, 만료일, 추적 정보가 들어 있음)


In [5]:
params = {'username': 'Ryan', 'password': 'password'}
r = requests.post("http://pythonscraping.com/pages/cookies/welcome.php", params)
print("Cookie is set to:")
print(r.cookies.get_dict())
print("-----------")
print("Going to profile page...")
r = requests.get("http://pythonscraping.com/pages/cookies/profile.php", cookies=r.cookies)
print(r.text)

Cookie is set to:
{'loggedin': '1', 'username': 'Ryan'}
-----------
Going to profile page...
Hey Ryan! Looks like you're still logged into the site!


### requests.Session : 가져온 세션 객체가 쿠키, 헤더, HTTPAdapters같은 HTTP 프로토콜에 관한 정보까지 관리합니다!

In [6]:
session = requests.Session()

params = {'username': 'username', 'password': 'password'}
s = session.post("http://pythonscraping.com/pages/cookies/welcome.php", params)
print("Cookie is set to:")
print(s.cookies.get_dict())
print("-----------")
print("Going to profile page...")
s = session.get("http://pythonscraping.com/pages/cookies/profile.php")
print(s.text)

Cookie is set to:
{'loggedin': '1', 'username': 'username'}
-----------
Going to profile page...
Hey username! Looks like you're still logged into the site!


### HTTP 기본 접근 인증
http://www.pythonscraping.com/pages/auth/login.php

- requests.auth 모듈을 사용-!

In [8]:
from requests.auth import AuthBase
from requests.auth import HTTPBasicAuth

auth = HTTPBasicAuth('ryan', 'password')
r = requests.post(url="http://pythonscraping.com/pages/auth/login.php", auth=auth)
print(r.text)

<p>Hello ryan.</p><p>You entered password as your password.</p>
